# Exercise 2

In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def negsel(dataset):
    """Determines the individual scores of all elements in dataset"""
    
    command = 'java -jar negsel2.jar -self syscalls/snd-cert/cert_train.txt -n 10 -r 5 -c -l < {}'.format(dataset)
    stream = os.popen(command)
    return stream.read()

In [3]:
def make_unique(sorted_list):
    """Filters out the items in the sorted list and labels that have duplicate scores"""
    unique_scores = []
    unique_list = []
    unique_labels = []
    for i, (s, t, l) in enumerate(sorted_list):
        if not s in unique_scores:
            unique_scores.append(s)
            unique_list.append(t)
            unique_labels.append(l)
    return unique_list, unique_labels

In [ ]:
# Load in scores
scores = negsel("syscalls/snd-cert/snd-cert.1.test")

# Format scores 
split_scores = [x.split() for x in scores.splitlines()]
# Take average score for lines longer than 10 characters
avg_scores = [np.average([float(x) for x in y]) for y in split_scores]

# Open test data and corresponding labels
test = open("syscalls/snd-cert/snd-cert.1.test").read().splitlines()
labels = open("syscalls/snd-cert/snd-cert.1.labels").read().splitlines()
labels = [int(x) for x in labels]

# Sort the test data based on the scores and remove duplicate scores
# N.B. sorted_list also contains the scores to be able to filter out the lines with duplicate scores
sorted_list = sorted(zip(avg_scores, test, labels))
unique_test, labels = make_unique(sorted_list)

In [ ]:
total_normal = len([x for x in labels if x == 0])
total_anom = len([x for x in labels if x == 1])
n_anom = total_anom
n_normal = 0
sensitivities = []
specificities = []

for i, term in enumerate(unique_test):
    sensitivities.append(n_anom / total_anom)
    specificities.append(1 - n_normal / total_normal)
    
    # Decrement n_anom if i == 1, increment n_normal if i == 0
    n_normal += (1 - labels[i])
    n_anom -= labels[i]

In [ ]:
# Plot
plt.figure(figsize=(10, 5))
plt.title("AUC curve for Unix processes with r = 5")
plt.plot(sensitivities, specificities)
plt.xlabel("Sensitivity")
plt.ylabel("Specificity")
plt.show()

In [ ]:
print(np.abs(np.trapz(specificities, sensitivities)))